#### I am crating this notebook since I realised that I should consolidate all the preprocessing of the data before training

In [22]:
import pandas as pd
import numpy as np
import os
import re
from ast import literal_eval

In [23]:
df = pd.read_csv('/Users/louishagenbucher/Documents/GitHub/mtg_nlp_price_prediction/data/cards_data.csv')
df.sample(5)

,id,name,cmc,mana_cost,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,textless,rarity,reserved,released_at,booster,usd
7981,6880a4d3-5cbc-4a01-9190-3565617efcc9,Flow of Maggots,3.0,{2}{B},['B'],False,['Creature'],['Insect'],Cumulative upkeep {1} (At the beginning of you...,['Cumulative upkeep'],2,2,False,rare,True,1995-06-03,True,0.97
11081,380e83c1-e5e3-49b2-bbf3-fad8cc7d020a,Initiate's Companion,2.0,{1}{G},['G'],False,['Creature'],['Cat'],Whenever Initiate's Companion deals combat dam...,['no keywords'],3,1,False,common,False,2020-07-17,True,0.03
3711,3969c209-09f3-4429-91c1-d5aeab9de2ec,Chromatic Lantern,3.0,{3},['C'],False,['Artifact'],['none'],"Lands you control have ""{T}: Add one mana of a...",['no keywords'],non-creature,non-creature,False,rare,False,2023-08-04,True,4.07
16060,24d6c178-ee0d-4907-bf6b-12b845bceaeb,Path to the World Tree,2.0,{1}{G},['G'],False,['Enchantment'],['none'],When Path to the World Tree enters the battlef...,['no keywords'],non-creature,non-creature,False,uncommon,False,2022-09-09,False,0.06
24581,2d7f565e-0fb8-40c8-9540-213d35af846a,Viridian Revel,3.0,{1}{G}{G},['G'],False,['Enchantment'],['none'],Whenever an artifact is put into an opponent's...,['no keywords'],non-creature,non-creature,False,uncommon,False,2010-10-01,True,1.02


In [24]:
df.drop(columns=['id', 'mana_cost', 'textless'], inplace=True)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26052 entries, 0 to 26051
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         26052 non-null  object 
 1   cmc          26052 non-null  float64
 2   colors       26052 non-null  object 
 3   legendary    26052 non-null  bool   
 4   types        26052 non-null  object 
 5   sub_types    26052 non-null  object 
 6   oracle_text  25099 non-null  object 
 7   keywords     26052 non-null  object 
 8   power        26052 non-null  object 
 9   toughness    26052 non-null  object 
 10  rarity       26052 non-null  object 
 11  reserved     26052 non-null  bool   
 12  released_at  26052 non-null  object 
 13  booster      26052 non-null  bool   
 14  usd          25996 non-null  float64
dtypes: bool(3), float64(2), object(10)
memory usage: 2.5+ MB


In [26]:
df['oracle_text'].fillna('notext', inplace=True)

In [27]:
df['sub_types'].fillna("['none']", inplace=True)

In [28]:
df['types'] = df['types'].apply(literal_eval)
df['sub_types'] = df['sub_types'].apply(literal_eval)
df['keywords'] = df['keywords'].apply(literal_eval)

In [29]:
df.drop(df[df['colors']=='[nan]'].index, inplace=True)
df.reset_index(drop=True)
df.head()

,name,cmc,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,rarity,reserved,released_at,booster,usd
0,+2 Mace,2.0,['W'],False,[Artifact],[Equipment],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,[Equip],non-creature,non-creature,common,False,2021-07-23,True,0.02
1,Aarakocra Sneak,4.0,['U'],False,[Creature],"[Bird, Rogue]",Flying\nWhen Aarakocra Sneak enters the battle...,[Flying],1,4,common,False,2022-06-10,True,0.06
2,Abaddon the Despoiler,5.0,"['B', 'R', 'U']",True,[Creature],"[Astartes, Warrior]",Trample\nMark of Chaos Ascendant — During your...,"[Mark of Chaos Ascendant, Trample]",5,5,mythic,False,2022-10-07,False,2.81
3,Abandoned Outpost,0.0,['No Colors'],False,[Land],[none],Abandoned Outpost enters the battlefield tappe...,[no keywords],non-creature,non-creature,common,False,2001-10-01,True,0.14
4,Abandoned Sarcophagus,3.0,['C'],False,[Artifact],[none],You may cast spells that have a cycling abilit...,[no keywords],non-creature,non-creature,rare,False,2020-04-17,False,0.09


In [30]:
df['colors'].replace("['No Colors']", "['N']", inplace=True)
df['colors']=df['colors'].apply(literal_eval)
df.reset_index(drop=True, inplace=True)

In [31]:
df['power'] = df['power'].replace('non-creature', 0)
df['toughness'] = df['toughness'].replace('non-creature', 0)

In [32]:
df.drop(df[df['types'].apply(lambda x: '//' in x)].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [33]:
df['power'] = pd.to_numeric(df['power'], errors='coerce')
df['toughness'] = pd.to_numeric(df['toughness'], errors='coerce')
df['power'] = df['power'].fillna(-1).astype(int)
df['toughness'] = df['toughness'].fillna(-1).astype(int)

In [34]:
df['released_at'] = pd.to_datetime(df['released_at'])
reference_date = df['released_at'].min()
df['released_at'] = (df['released_at'] - reference_date).dt.days

In [35]:
df['cmc'] = df['cmc'].astype(int)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25545 entries, 0 to 25544
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         25545 non-null  object 
 1   cmc          25545 non-null  int64  
 2   colors       25545 non-null  object 
 3   legendary    25545 non-null  bool   
 4   types        25545 non-null  object 
 5   sub_types    25545 non-null  object 
 6   oracle_text  25545 non-null  object 
 7   keywords     25545 non-null  object 
 8   power        25545 non-null  int64  
 9   toughness    25545 non-null  int64  
 10  rarity       25545 non-null  object 
 11  reserved     25545 non-null  bool   
 12  released_at  25545 non-null  int64  
 13  booster      25545 non-null  bool   
 14  usd          25489 non-null  float64
dtypes: bool(3), float64(1), int64(4), object(7)
memory usage: 2.4+ MB


In [37]:
directory_path = '/Users/louishagenbucher/Documents/GitHub/mtg_nlp_price_prediction/data'
df.to_csv(os.path.join(directory_path, 'clean_cards.csv'), index=False)